In [5]:
# import numpy as np
# import torch
# import warnings
# from copy.deepcopy(Munch import copy.deepcopy(Munch
# import itertools
# from mv_Viterbi import mv_Viterbi
# from cst_aggregate import cst_aggregate

import numpy as np
import torch
import json
from munch import Munch
import itertools
from collections import defaultdict
import random
import copy
import pickle
import matplotlib.pyplot as plt
import importlib
import time


### Create the HMM

In [6]:
hidden_states = ['pro','ex1','ex2','int','dis','enh']
emit_states = ['A','T', 'C', 'G']
hidden_size, emit_size = len(hidden_states), len(emit_states)

hmm_mat = np.array([
    [.6,.1,.1,.2,0,0], #promoter
    [0,.4,.2,.2,.1,.1], #exon1
    [.0,.1,.6,.1,.1,.1], #exon2
    [.3,.1,.1,.4,0,.1], #intron
    [0,1/3, 1/3, 0,1/3,0], #disease
    [0,.25,.25,.25,0,.25] #enhancer
])

emit_mat = np.array([ #
    [.1,.1,.4,.4], #CG rich promoter
    [.2,.2,.5,.1], #Exon 1 favors C
    [.5,.1,.2,.2], #Exon 2 favors A
    [.25,.25,.25,.25], #Intron 
    [.4,.1,.4,.1], #Disease favors AC
    [.4,.4,.1,.1] #AT rich enhancer
])

init_vec = np.array(
    [.2,0,0,.8,0,0]
)

hmm_transition = {}
for i in range(hidden_size):
    for j in range(hidden_size):
        hmm_transition[hidden_states[i],hidden_states[j]] = hmm_mat[i,j].item()

hmm_emit = {}
for i in range(hidden_size):
    for j in range(emit_size):
        hmm_emit[hidden_states[i],emit_states[j]] = emit_mat[i,j].item()
        
hmm_startprob = {}
for i in range(hidden_size):
    hmm_startprob[hidden_states[i]] = init_vec[i]

hmm = copy.deepcopy(Munch(states = hidden_states, emits = emit_states, tprob = hmm_transition, eprob = hmm_emit, initprob = hmm_startprob))

In [7]:
def create_cst_params(cst, hidden_states, dtype = torch.float32, device = 'cpu'):
    m_states = cst.m_states
    init = cst.init_fun
    upd = cst.update_fun
    eval_fun = cst.eval_fun

    #returns a (k,s,r) array. k is current hideen. r,s are present/past mediation.
    upd_mat = torch.tensor([[[upd(k,r,s) for s in m_states] for r in m_states] for k in hidden_states], dtype = dtype, device = device)

    #returns a (k,r) array. k,r are current hidden/mediation states
    init_mat = torch.tensor([[init(k,r) for r in m_states] for k in hidden_states], dtype = dtype, device = device)

    #return (k,r) array for terminal emission.
    eval_mat = torch.tensor([[eval_fun(k,r) for r in m_states] for k in hidden_states], dtype = dtype, device = device)

    return init_mat, eval_mat, upd_mat


### Stay > = 3

In [8]:
def update_fun(k , r, r_past):
    '''
    r = hidden_states x [1,2,3]
    '''
    prev, count = r_past #r is a tuple
    if k == prev:
        new_count = min(count + 1 , 3)
    else:
        new_count = 1
        
    consistency = (count == 3) or (k == prev) #0 if transition to new state without staying 3

    if (count == 3 or k == prev) and r == (k, new_count):
        return True
    else:
        return False

def init_fun(k, r):
    '''
    initial "prob" of r = (m1,m2) from k. is just indicator
    '''

    return r == (k,1)

    
def eval_fun(k, r):
    return 1

m_states = list(itertools.product(hidden_states, list(range(1,4))))

stay_cst = Munch(update_fun = update_fun, init_fun = init_fun, eval_fun = eval_fun, m_states = m_states)

In [9]:
update_fun('enh',('pro',4), ('enh',1))

False

In [10]:
test_params = create_cst_params(stay_cst, hidden_states)

print(test_params[0].sum(axis = 1),test_params[1].sum(axis = 1),test_params[2].sum(axis = 1))

tensor([1., 1., 1., 1., 1., 1.]) tensor([18., 18., 18., 18., 18., 18.]) tensor([[1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1.],
        [0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1.]])


### Promoter Must Occur in First 30

In [11]:
def update_fun(k , r, r_past):
    '''
    r = Boolean
    tracks if 'pro' has occured yet or not
    '''        

    return r == (r_past or (k == 'pro')) 

def init_fun(k, r):

    return r == (k == 'pro')

def eval_fun(k,r):
    return 1

m_states = [True,False]

promoter_cst = copy.deepcopy(Munch(update_fun = update_fun, init_fun = init_fun, eval_fun = eval_fun, m_states = m_states))

In [12]:
test_params = create_cst_params(promoter_cst, hidden_states)

print(test_params[0].sum(axis = 1),test_params[1].sum(axis = 1),test_params[2].sum(axis = 1))

tensor([1., 1., 1., 1., 1., 1.]) tensor([2., 2., 2., 2., 2., 2.]) tensor([[1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.],
        [1., 1.]])


#### Visit Dis Exactly Once 

In [13]:
def update_fun(k , r, r_past):
    '''
    r =  [True,False] x [0,1,2]
    r[0] is True if previoius state was NOT dis.
    counts number of entries into dis (ie. number of dis regions)
    '''
    prev, count = r_past

    if prev and k == 'dis':
        count = min(count + 1, 2)    
    
    return r == (k != 'dis',count) 

def init_fun(k, r):

    if k == 'dis':
        return r == (False,1)
    else:
        return r == (True,0)

def eval_fun(k,r):
    return r[1] == 1 #must be exactly 1.

m_states = list(itertools.product([True,False], list(range(3))))


disvisit_cst = copy.deepcopy(Munch(update_fun = update_fun, init_fun = init_fun, eval_fun = eval_fun, m_states = m_states))

### Promoter < Disease < Enhancer

In [14]:
def update_fun(k , r, r_past):
    '''
    r = Boolean_pro x Bool_dis x Bool_enh
    trcks that they occur in sequence
    '''
    occur_pro, occur_dis, occur_enh = r_past
    consist = True
    
    pro_new = (k == 'pro' or occur_pro)
    dis_new = (k == 'dis' or occur_dis)
    enh_new = (k == 'enh' or occur_enh)

    if k == 'dis': #if at anytime dis occurs before pro, that sequence is zero'd out.
        consist = pro_new

    if k == 'enh':
        consist = dis_new 

    return (r == (pro_new, dis_new,enh_new)) and consist

def init_fun(k, r):

    return r == ( k == 'pro', k == 'dis', k == 'enh')

def eval_fun(k,r):
    return r[2] #ensures that enhancer is in the sequence

m_states = list(itertools.product([True, False], repeat=3))

pde_cst = copy.deepcopy(Munch(update_fun = update_fun, init_fun = init_fun, eval_fun = eval_fun, m_states = m_states))



# Dummy Constraint

In [15]:
def update_fun(k, r, r_past):
    return r == True

def init_fun(k, r):

    return r == True

def eval_fun(k, r):
    return 1

m_states = [True,False]

dummy_cst = copy.deepcopy(Munch(update_fun = update_fun, init_fun = init_fun, eval_fun = eval_fun, m_states = m_states))


In [16]:
def create_cst_params(cst, hidden_states, dtype = torch.float32, device = 'cpu'):
    m_states = cst.m_states
    init = cst.init_fun
    upd = cst.update_fun
    eval_fun = cst.eval_fun

    #returns a (k,s,r) array. k is current hideen. r,s are present/past mediation.
    upd_mat = torch.tensor([[[upd(k,r,s) for s in m_states] for r in m_states] for k in hidden_states], dtype = dtype, device = device)

    #returns a (k,r) array. k,r are current hidden/mediation states
    init_mat = torch.tensor([[init(k,r) for r in m_states] for k in hidden_states], dtype = dtype, device = device)

    #return (k,r) array for terminal emission.
    eval_mat = torch.tensor([[eval_fun(k,r) for r in m_states] for k in hidden_states], dtype = dtype, device = device)

    return init_mat, eval_mat, upd_mat

def convertTensor_list(hmm, cst_list, dtype = torch.float16, device = 'cpu', hmm_params = None, return_ix = False):
    '''
    cst_list is a list of the individual csts.
    '''
    #Initialize and convert all quantities  to np.arrays
    hmm = copy.deepcopy(hmm)
    K = len(hmm.states)
    
    state_ix = {s: i for i, s in enumerate(hmm.states)}
    
    #Compute the hmm parameters if not provided
    if hmm_params is None:
        tmat = torch.zeros((K,K), dtype=dtype ).to(device)
        init_prob = torch.zeros(K, dtype=dtype ).to(device)
    
        for i in hmm.states:
            init_prob[state_ix[i]] = hmm.initprob[i]
            for j in hmm.states:
                tmat[state_ix[i],state_ix[j]] = hmm.tprob[i,j]
    
        hmm_params = [tmat, init_prob]
    
    #Compute the cst parameters 
    init_list = []
    eval_list = []
    upd_list = []
    dims_list = []
    cst_ix = 0
    C = len(cst_list)

    #indices are (hidden, c_1,....,c_C, hidden, c_1,....,c_C) are augmented messages
    for cst in cst_list:
        cst = copy.deepcopy(cst)
        init_mat, eval_mat, upd_mat = create_cst_params(cst, hidden_states, dtype = dtype, device = device)
        init_list += [init_mat,[0,cst_ix + 1]]
        eval_list += [eval_mat, [0, cst_ix + 1]]
        upd_list += [upd_mat, [0, cst_ix + 1,cst_ix + C + 2]]
        dims_list.append(len(cst.m_states))
        cst_ix += 1
                
    cst_params = [dims_list, init_list,eval_list,upd_list]

    if return_ix:
        return hmm_params, cst_params, state_ix
    return hmm_params, cst_params 


In [17]:
dummy_list = [dummy_cst]

In [18]:
_, dummy_params = convertTensor_list(hmm, dummy_list, dtype = torch.float16, device = 'cpu', hmm_params = None, return_ix = False)

In [19]:
def compute_emitweights(obs,hmm):
    '''
    Separately handles the computation of the 
    '''
    hmm = copy.deepcopy(hmm) #protect again in place modification
    T = len(obs)
    K = len(hmm.states)
    #Compute emissions weights for easier access
    emit_weights = np.zeros((T,K))
    for t in range(T):
        emit_weights[t] = np.array([hmm.eprob[k,obs[t]] for k in hmm.states])
    return emit_weights


 # Simulation Code


In [20]:
def hmm2numpy(hmm, ix_list = None, return_ix = False):
    '''
    Converts/generates relevant parameters/weights into numpy arrays for Baum-Welch.
    By assumption, the update/emission parameters associated with the constraint are static.
    For now, fix the emission probabilities.
    Only the hmm paramters are being optimized.
    '''
    #Initialize and convert all quantities  to np.arrays

    if ix_list:
        state_ix, emit_ix = ix_list
    else:
        state_ix = {s: i for i, s in enumerate(hmm.states)}
        emit_ix = {s: i for i, s in enumerate(hmm.emits)}

    K = len(state_ix)
    M = len(emit_ix)
    #Compute the hmm parameters
    tmat = np.zeros((K,K))
    init_prob = np.zeros(K)

    emat = np.zeros((K,M))

    #Initial distribution. 
    for i in hmm.states:
        if i not in hmm.initprob:
            continue
        init_prob[state_ix[i]] = hmm.initprob[i]

    #Transition matrix
    for i in hmm.states:
        for j in hmm.states:
            if (i,j) not in hmm.tprob:
                continue
            tmat[state_ix[i],state_ix[j]] = hmm.tprob[i,j]

    
    #Emission matrix
    for i in hmm.states:
        for m in hmm.emits:
            if (i,m) not in hmm.eprob:
                continue
            emat[state_ix[i],emit_ix[m]] = hmm.eprob[i,m]

    hmm_params = [init_prob, tmat, emat]

    if return_ix:
        return hmm_params, [state_ix, emit_ix] 
    return hmm_params


In [21]:
def random_draw(p):
    '''
    p is a 1D np array. 
    single random draw from probability vector p and encode as 1-hot.
    '''
    n = len(p)
    p = p/p.sum()
    draw = np.random.choice(n,p=p)
    one_hot = np.zeros(n, dtype = int)
    one_hot[draw] = 1
    
    return one_hot

In [27]:
def single_simulation(hmm, time, stay=5, pro_before= 30, ix_list=None):
    '''
    Draws from hmm with addition constraint that we stay in each state for at least duration "stay"
    pro_before sets the maximum time horizon that promoter must occur by.
    '''
    # Get numpy version of hmm parameters
    hmm_params, ix_list = hmm2numpy(hmm, ix_list=ix_list, return_ix=True) 
    init_prob, tmat, emat = hmm_params

    # Prepare dictionary for converting one_hot back to states
    state_ix, emit_ix = ix_list
    state_ix = {v: k for k, v in state_ix.items()}
    emit_ix = {v: k for k, v in emit_ix.items()}


    # Generate (X1,Y1)
    x_curr = random_draw(init_prob)
    current_state = state_ix[np.argmax(x_curr)] # convert one-hot back to state
    x_list = [current_state] 
    emit_dist = x_curr @ emat
    y_curr = random_draw(emit_dist)
    y_list = [emit_ix[np.argmax(y_curr)]]

    x_prev = x_curr

    #Initialize visit_trackers
    visit_pro = current_state == 'pro'
    visit_dis = current_state == 'dis'
    visit_enh = current_state == 'enh'

    dis_visits = int(current_state == 'dis')
    
    # Initialize state stay counter
    stay_counter = 1

    
    # Generate rest
    for t in range(1, time):
        # By Markov property, just clamp to current stay until stay for required time
        if stay_counter < stay:
            stay_counter += 1
        else:
            # Transition to a new state
            x_curr = random_draw(x_prev @ tmat)
            if np.argmax(x_prev) != np.argmax(x_curr):
                stay_counter = 1  # Reset stay counter for the new state
                current_state = state_ix[np.argmax(x_curr)]
                emit_dist = x_prev @ emat

                x_prev = x_curr


                #Update visit_trackers
                visit_pro = visit_pro or current_state == 'pro'
                visit_dis = visit_dis or current_state == 'dis'
                visit_enh = visit_enh or current_state == 'enh'

                if current_state == 'dis':
                #this condition already assumes transition to new state, so records new dis region.
                    dis_visits += 1

        #Constraints
         #check we hit promoter by pro_before 
        if t == int(pro_before) and (not visit_pro):
            return False

        #pro < dis < enh

        if (not visit_pro) and visit_dis:
            return False

        if (not visit_dis) and visit_enh:
            return False

        y_curr = random_draw(emit_dist)
        
        x_list.append(current_state)
        y_list.append(emit_ix[np.argmax(y_curr)])

    #check if only one dis region
    if dis_visits != 1:
        return False
        
    return x_list, y_list

def simulation(hmm, time, stay=5, pro_before=30, ix_list=None, max_attempts=1000):
    '''
    Repeatedly calls the simulation function until a valid full run is generated.
    Returns the first valid simulation (list of states and emissions).
    If no valid simulation is found within max_attempts, raises an exception.
    '''
    for attempt in range(max_attempts):
        result = single_simulation(hmm, time, stay=stay, pro_before=pro_before, ix_list=ix_list)
        if result is not False:
            return result  # Return the valid simulation

    raise RuntimeError(f"Failed to generate a valid simulation after {max_attempts} attempts.")


In [28]:
def Viterbi_torch_list(hmm, cst_list, obs, pro_before = 30, dtype = torch.float32,  device = 'cpu', debug = False, num_corr = 0, hmm_params = None):
    '''
    more optimized torch implementation of Viterbi. The constraint all evolve independently (ie. factorial), so no need to create a big U_krjs matrix. Instead, just multiply along given dim. Still require computing V_{krjs}, but this should help.
    For numerica underflow, we normalize the value at each time. Also, we add a small constant num_corr when normalizing.

    For DNA, always assume that the promoter constraint is first.
    '''
    hmm = copy.deepcopy(hmm) #protect again in place modification
    #Generate emit_weights:
    emit_weights = compute_emitweights(obs, hmm)
    emit_weights = torch.from_numpy(emit_weights).type(dtype).to(device)

    #Generate hmm,cst params:
    hmm_params, cst_params_list, state_ix = convertTensor_list(hmm,cst_list, dtype = dtype, \
                                                               device = device, return_ix = True, hmm_params = hmm_params)   
    tmat, init_prob = hmm_params
    dims_list, init_ind_list,final_ind_list,ind_list = cst_params_list

    
    #Viterbi
    T = emit_weights.shape[0]
    K = tmat.shape[0]
    C = len(dims_list)
    
    val = torch.empty((T,K) + tuple(dims_list), device = 'cpu')
    ix_tracker = torch.empty((T,K) + tuple(dims_list), device = 'cpu') #will store flattened indices
    
    kr_indices = list(range(C+1))
    kr_shape = (K,) + tuple(dims_list)
    js_indices = [k + C + 1 for k in kr_indices]

    #Forward pass
    # V = torch.einsum('k,k,kr -> kr', init_prob, emit_weights[0], init_ind)
    V = torch.einsum(emit_weights[0], [0], init_prob, [0], *init_ind_list, kr_indices)
    V = V/(V.max() + num_corr) #normalize for numerical stability
    val[0] = V.cpu()
    
    for t in range(1,T):
        # return kr_indices, ind_list, dims_list, C
        # V = torch.einsum('js,jk,krjs -> krjs',val[t-1],tmat,ind)
        V = torch.einsum(val[t-1].to(device), js_indices, tmat, [C+1,0], *ind_list, list(range(2*C + 2)))
        V = V.reshape(tuple(kr_shape) + (-1,))
        V = V/(V.max() + num_corr)
        max_ix = torch.argmax(V, axis = -1, keepdims = True)
        ix_tracker[t-1] = max_ix.squeeze()
        V = torch.take_along_dim(V, max_ix, axis=-1).squeeze()
        # if t == T:
        #     # val[t] = torch.einsum('k,kr,kr -> kr',emit_weights[t],final_ind,V)
        #     val[t] = torch.einsum(emit_weights[t],[0], V, kr_indices,*final_ind_list, kr_indices).cpu()
        # else:
        #     # val[t] = torch.einsum('k,kr -> kr', emit_weights[t],V)
        #     val[t] = torch.einsum(emit_weights[t],[0], V, kr_indices, kr_indices).cpu()
        if t == pro_before:
            # Evaluate only the first constraint at time = 30
            val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, *final_ind_list[:2], kr_indices).cpu()
        elif t == T -1:
            # Evaluate all constraints at the last time
            val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, *final_ind_list[2:], kr_indices).cpu()
        else:
            # Regular update without evaluating constraints
            val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, kr_indices).cpu()


    # return val
    state_ix = {v:k for k,v in state_ix.items()}
    #Backward pass
    opt_augstateix_list = []
    max_ix = int(torch.argmax(val[T-1]).item())
    unravel_max_ix = np.unravel_index(max_ix, kr_shape)
    opt_augstateix_list =  [np.array(unravel_max_ix).tolist()] + opt_augstateix_list
    
    ix_tracker = ix_tracker.reshape(T,-1) #flatten again for easier indexing    

    for t in range(T-1):
        max_ix =  int(ix_tracker[T-2-t,max_ix].item())
        unravel_max_ix = np.unravel_index(max_ix, kr_shape)
        opt_augstateix_list =  [np.array(unravel_max_ix).tolist()] + opt_augstateix_list

    opt_state_list = [state_ix[k[0]] for k in opt_augstateix_list]
    if debug:
        return opt_state_list, opt_augstateix_list, val, ix_tracker
    return opt_state_list, opt_augstateix_list


In [29]:
state_list, emit_list = simulation(hmm, time = 100, stay=5, pro_before=30, ix_list=None, max_attempts=1000)

# Experiments

In [31]:
state_list, emit_list = simulation(hmm, time = 30, stay=3, pro_before=10, ix_list=None, max_attempts=1000)

In [32]:
cst_list = [promoter_cst,stay_cst,disvisit_cst, pde_cst]

In [33]:
dummy_list = [dummy_cst]

In [34]:
opt_state_list, opt_augstateix_list = Viterbi_torch_list(hmm, cst_list, emit_list, pro_before = 30, dtype = torch.float32,  device = 'cpu', debug = False, num_corr = 0)

In [38]:
list(zip(opt_state_list,state_list))

[('int', 'int'),
 ('int', 'int'),
 ('int', 'int'),
 ('int', 'int'),
 ('int', 'pro'),
 ('int', 'pro'),
 ('int', 'pro'),
 ('pro', 'int'),
 ('pro', 'int'),
 ('pro', 'int'),
 ('pro', 'pro'),
 ('pro', 'pro'),
 ('pro', 'pro'),
 ('pro', 'int'),
 ('pro', 'int'),
 ('pro', 'int'),
 ('pro', 'ex1'),
 ('ex2', 'ex1'),
 ('ex2', 'ex1'),
 ('ex2', 'ex1'),
 ('ex2', 'dis'),
 ('dis', 'dis'),
 ('dis', 'dis'),
 ('dis', 'ex1'),
 ('ex1', 'ex1'),
 ('ex1', 'ex1'),
 ('ex1', 'ex2'),
 ('enh', 'ex2'),
 ('enh', 'ex2'),
 ('enh', 'ex2')]

In [594]:
cst_list = [promoter_cst, stay_cst, pde_cst]

In [603]:
val = Viterbi_torch_list(hmm, cst_list, emit_list, pro_before = 30, dtype = torch.float32,  device = 'cpu', debug = False, num_corr = 0)

In [604]:
for t in range(50):
    print(val[t].max())

tensor(1.)
tensor(0.2500)
tensor(0.2500)
tensor(0.3000)
tensor(0.1389)
tensor(0.4000)
tensor(0.1000)
tensor(0.2083)
tensor(0.2000)
tensor(0.3840)
tensor(0.4000)
tensor(0.1302)
tensor(0.1067)
tensor(0.5000)
tensor(0.2000)
tensor(0.1000)
tensor(0.2000)
tensor(0.2844)
tensor(0.3516)
tensor(0.5000)
tensor(0.5000)
tensor(0.5000)
tensor(0.5000)
tensor(0.2000)
tensor(0.2000)
tensor(0.5000)
tensor(0.5000)
tensor(0.2000)
tensor(0.2000)
tensor(0.1000)
tensor(0.5000)
tensor(0.5000)
tensor(0.2000)
tensor(0.2000)
tensor(0.5000)
tensor(0.2000)
tensor(0.5000)
tensor(0.1000)
tensor(0.1000)
tensor(0.2000)
tensor(0.2000)
tensor(0.2315)
tensor(0.1000)
tensor(0.4000)
tensor(0.4000)
tensor(0.1080)
tensor(0.3704)
tensor(0.2700)
tensor(0.2000)
tensor(0.0129)


In [383]:
obs = emit_list
dtype = torch.float32
device = 'cpu'
hmm_params = None


hmm = copy.deepcopy(hmm) #protect again in place modification
#Generate emit_weights:
emit_weights = compute_emitweights(obs, hmm)
emit_weights = torch.from_numpy(emit_weights).type(dtype).to(device)

#Generate hmm,cst params:
hmm_params, cst_params_list, state_ix = convertTensor_list(hmm,cst_list, dtype = dtype, \
                                                           device = device, return_ix = True, hmm_params = hmm_params)   
tmat, init_prob = hmm_params
dims_list, init_ind_list,final_ind_list,ind_list = cst_params_list



In [384]:
for i in range(4):
    print(ind_list[2*i].shape, ind_list[2*i + 1])

torch.Size([6, 2, 2]) [0, 1, 6]
torch.Size([6, 18, 18]) [0, 2, 7]
torch.Size([6, 3, 3]) [0, 3, 8]
torch.Size([6, 8, 8]) [0, 4, 9]


In [466]:
final_ind_list[0].shape

torch.Size([6, 2])

In [465]:
final_ind_list[1]

[0, 1]

In [448]:
    #Viterbi
    T = emit_weights.shape[0]
    K = tmat.shape[0]
    C = len(dims_list)
    
    val = torch.empty((T,K) + tuple(dims_list), device = 'cpu')
    ix_tracker = torch.empty((T,K) + tuple(dims_list), device = 'cpu') #will store flattened indices
    
    kr_indices = list(range(C+1))
    kr_shape = (K,) + tuple(dims_list)
    js_indices = [k + C + 1 for k in kr_indices]


In [449]:
num_corr = 0
V = torch.einsum(emit_weights[0], [0], init_prob, [0], *init_ind_list, kr_indices)
V = V/(V.max() + num_corr) #normalize for numerical stability
val[0] = V.cpu()


In [462]:
pro_before = 30
for t in range(1,T):
    # return kr_indices, ind_list, dims_list, C
    # V = torch.einsum('js,jk,krjs -> krjs',val[t-1],tmat,ind)
    V = torch.einsum(val[t-1].to(device), js_indices, tmat, [C+1,0], *ind_list, list(range(2*C + 2)))
    V = V.reshape(tuple(kr_shape) + (-1,))
    V = V/(V.max() + num_corr)
    max_ix = torch.argmax(V, axis = -1, keepdims = True)
    ix_tracker[t-1] = max_ix.squeeze()
    V = torch.take_along_dim(V, max_ix, axis=-1).squeeze()
    # if t == T:
    #     # val[t] = torch.einsum('k,kr,kr -> kr',emit_weights[t],final_ind,V)
    #     val[t] = torch.einsum(emit_weights[t],[0], V, kr_indices,*final_ind_list, kr_indices).cpu()
    # else:
    #     # val[t] = torch.einsum('k,kr -> kr', emit_weights[t],V)
    #     val[t] = torch.einsum(emit_weights[t],[0], V, kr_indices, kr_indices).cpu()
    if t == T -1:
        # Evaluate all constraints at the last time
        val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, *final_ind_list[2:], kr_indices).cpu()
    
    elif t == pro_before:
        # Evaluate only the first constraint at time = 30
        val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, *final_ind_list[:2], kr_indices).cpu()

    else:
        # Regular update without evaluating constraints
        val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, kr_indices).cpu()


In [463]:
for t in range(50):
    print(val[t].max())

tensor(1.)
tensor(0.2500)
tensor(0.4000)
tensor(0.4000)
tensor(0.1000)
tensor(0.4000)
tensor(0.4000)
tensor(0.1000)
tensor(0.4000)
tensor(0.4000)
tensor(0.1000)
tensor(0.4000)
tensor(0.4000)
tensor(0.1042)
tensor(0.1333)
tensor(0.3840)
tensor(0.1000)
tensor(0.1608)
tensor(0.2500)
tensor(0.2500)
tensor(0.3000)
tensor(0.4000)
tensor(0.1000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.1000)
tensor(0.1389)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.1000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.4000)
tensor(0.1000)
tensor(0.1389)
tensor(0.4000)
tensor(0.4000)
tensor(0.1042)
tensor(0.3840)
tensor(0.1000)
tensor(0.)


In [441]:
def Viterbi_torch_list(hmm, cst_list, obs, pro_before = 30, dtype = torch.float32,  device = 'cpu', debug = False, num_corr = 0, hmm_params = None):
    '''
    more optimized torch implementation of Viterbi. The constraint all evolve independently (ie. factorial), so no need to create a big U_krjs matrix. Instead, just multiply along given dim. Still require computing V_{krjs}, but this should help.
    For numerica underflow, we normalize the value at each time. Also, we add a small constant num_corr when normalizing.

    For DNA, always assume that the promoter constraint is first.
    '''
    hmm = copy.deepcopy(hmm) #protect again in place modification
    #Generate emit_weights:
    emit_weights = compute_emitweights(obs, hmm)
    emit_weights = torch.from_numpy(emit_weights).type(dtype).to(device)

    #Generate hmm,cst params:
    hmm_params, cst_params_list, state_ix = convertTensor_list(hmm,cst_list, dtype = dtype, \
                                                               device = device, return_ix = True, hmm_params = hmm_params)   
    tmat, init_prob = hmm_params
    dims_list, init_ind_list,final_ind_list,ind_list = cst_params_list

    
    #Viterbi
    T = emit_weights.shape[0]
    K = tmat.shape[0]
    C = len(dims_list)
    
    val = torch.empty((T,K) + tuple(dims_list), device = 'cpu')
    ix_tracker = torch.empty((T,K) + tuple(dims_list), device = 'cpu') #will store flattened indices
    
    kr_indices = list(range(C+1))
    kr_shape = (K,) + tuple(dims_list)
    js_indices = [k + C + 1 for k in kr_indices]

    #Forward pass
    # V = torch.einsum('k,k,kr -> kr', init_prob, emit_weights[0], init_ind)
    V = torch.einsum(emit_weights[0], [0], init_prob, [0], *init_ind_list, kr_indices)
    V = V/(V.max() + num_corr) #normalize for numerical stability
    val[0] = V.cpu()
    
    for t in range(1,T):
        # return kr_indices, ind_list, dims_list, C
        # V = torch.einsum('js,jk,krjs -> krjs',val[t-1],tmat,ind)
        V = torch.einsum(val[t-1].to(device), js_indices, tmat, [C+1,0], *ind_list, list(range(2*C + 2)))
        V = V.reshape(tuple(kr_shape) + (-1,))
        V = V/(V.max() + num_corr)
        max_ix = torch.argmax(V, axis = -1, keepdims = True)
        ix_tracker[t-1] = max_ix.squeeze()
        V = torch.take_along_dim(V, max_ix, axis=-1).squeeze()
        # if t == T:
        #     # val[t] = torch.einsum('k,kr,kr -> kr',emit_weights[t],final_ind,V)
        #     val[t] = torch.einsum(emit_weights[t],[0], V, kr_indices,*final_ind_list, kr_indices).cpu()
        # else:
        #     # val[t] = torch.einsum('k,kr -> kr', emit_weights[t],V)
        #     val[t] = torch.einsum(emit_weights[t],[0], V, kr_indices, kr_indices).cpu()
        if t == T -1:
            # Evaluate all constraints at the last time
            val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, *final_ind_list[2:], kr_indices).cpu()
        
        # elif t == pro_before:
        #     # Evaluate only the first constraint at time = 30
        #     val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, *final_ind_list[:2], kr_indices).cpu()

        else:
            # Regular update without evaluating constraints
            val[t] = torch.einsum(emit_weights[t], [0], V, kr_indices, kr_indices).cpu()

        
    state_ix = {v:k for k,v in state_ix.items()}
    #Backward pass
    opt_augstateix_list = []
    max_ix = int(torch.argmax(val[T-1]).item())
    unravel_max_ix = np.unravel_index(max_ix, kr_shape)
    opt_augstateix_list =  [np.array(unravel_max_ix).tolist()] + opt_augstateix_list
    
    ix_tracker = ix_tracker.reshape(T,-1) #flatten again for easier indexing    

    
    for t in range(T-1):
        max_ix =  int(ix_tracker[T-2-t,max_ix].item())
        unravel_max_ix = np.unravel_index(max_ix, kr_shape)
        opt_augstateix_list =  [np.array(unravel_max_ix).tolist()] + opt_augstateix_list

    opt_state_list = [state_ix[k[0]] for k in opt_augstateix_list]
    if debug:
        return opt_state_list, opt_augstateix_list, val, ix_tracker
    return opt_state_list, opt_augstateix_list
